In [6]:
include("../LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using HDF5
using DynamicPolynomials
using Plots
using Dates

using Optim
using Random

# Define polynomial variables
@polyvar w a

function get_pop_lme_obj(gamma)
    
    obj_file="../MODELS/POP_objectives_simple_2024-Jan-13_at_11-03.h5"

    h5open(obj_file,"r") do fid # read file, preserve existing content
        polynomial_str = read(fid["gamma_"*gamma]["pop_lme_objective"]) 
        # Parse the string into a DynamicPolynomial
        #@polyvar w a
        poly = eval(Meta.parse(polynomial_str))
        return(poly)
    end
end

function scaling_poly(p::Polynomial)

    X = transpose(hcat([exponents(t) for t in terms(p)]...))

    # Get the scaling via linear regression
    scaling = X \ log.(abs.(coefficients(p)))

    exp.(abs.(scaling)) # 
end

function local_rand_min(p)

    pd = p / maximum(abs.(coefficients(p)))

    # find variable scaling
    scale = scaling_poly(pd)

    # scale the polynomial
    p_scaled = subs(pd, variables(pd) => scale .* variables(pd))

    num_iterations = 100

    # Initialize the best minimizer and the minimum value
    best_minimizer = nothing
    best_min_value = Inf

    num_of_variables = length(variables(pd))

    for _ in 1:num_iterations
        # Generate a random initial condition
        initial_point = rand(num_of_variables).*250

        # Run local optimization
        result = Optim.optimize(p_scaled, initial_point, BFGS())
        #println(Optim.minimum(result))

        # Update the best minimizer if a better one is found
        if Optim.minimum(result) < best_min_value
            
            best_minimizer = Optim.minimizer(result)
            best_min_value = Optim.minimum(result)
            
        end

    end

    best_minimizer = abs.(best_minimizer) # to make gamma positive

    minimizer_scaled = scale .* best_minimizer

    solution = variables(p_scaled) => minimizer_scaled

end


local_rand_min (generic function with 1 method)

In [7]:
data_dir = "../DATA/"
println(data_dir)

models_dir = "../MODELS/"
tests_dir = "../TESTS/"

dodeca_files = ["State_D"*string(n) for n=1:20];

basis_files = ["State_B"*string(n) for n=1:4];

all_files = vcat(dodeca_files, basis_files)
train_files = basis_files 
test_files = dodeca_files

Hˢʸᵐᵇ = [ w   0
          0   0. ]

Jˢʸᵐᵇ = [ 0   a
          0   0. ]


../DATA/


2×2 Matrix{Term{Float64, Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}}:
 0.0  a
 0.0  0.0

In [11]:
γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

tests_data_file_name = "POP_LME_random_trn4_tst20_"*date_and_time_string * ".h5"

for γᵢ in γ

    println("γ =  "*γᵢ)

    poly = get_pop_lme_obj(γᵢ)
    
    sol = local_rand_min(poly)

    Hˢⁱᵈ = subs(Hˢʸᵐᵇ, sol)
    Jˢⁱᵈ = subs(Jˢʸᵐᵇ, sol)

    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ) # create coupling group        
    end

    for df in test_files # loop over initial states
        
        print(df*" ")

        start_time = time()

        ρₛ, tₛ = LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)

        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        tˢⁱᵈ, ρˢⁱᵈ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈ), [DenseOperator(basis,Jˢⁱᵈ)])

        Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]                    

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group

            init_state_group = create_group(γ_group, df) # create initial state group
            init_state_group["F_lme_exact"] = convert.(Float64, Fᴸᴹᴱₑₓ)
            
        end
                    
        println(minimum(Fᴸᴹᴱₑₓ))
    
    end

    println()
end

println(tests_data_file_name)

γ =  0.079477


State_D1 0.9993686181081669


State_D2 

0.9993718455675805
State_D3 0.9993717278675478


State_D4 

...

0.9991036749640971
State_D7 0.9925934960313113
State_D8 0.9957393886187114
State_D9 0.9985565392616734
State_D10 0.9992664886141212
State_D11 0.9965616147198867
State_D12 

0.9990294707074874
State_D13 0.996015730758893
State_D14 0.9985949429950067
State_D15 0.9993414241490192
State_D16 0.9993414241489537
State_D17 0.9925934960309484


State_D18 0.9957393886186215
State_D19 0.999266488614124
State_D20 0.9985565392615954

γ =  251.33
State_D1 0.9965356821318476
State_D2 0.9951092850043142
State_D3 0.9957899341833354
State_D4 0.9944258961187287


State_D5 0.9934629396414179
State_D6 0.9934629396414179
State_D7 0.9996798923877068
State_D8 0.998272716387193
State_D9 0.9947920896444131
State_D10 0.9936328461526904
State_D11 0.9957899341833221
State_D12 0.9944258961187287
State_D13 0.9965356821318476
State_D14 0.9951092850043781


State_D15 0.9930190594861086
State_D16 0.993019059486061
State_D17 0.9996798923877179
State_D18 0.9982727163871887
State_D19 0.9936328461526904
State_D20 0.9947920896443051

POP_LME_random_trn4_tst20_2024-Jan-14_at_12-03.h5
